In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

# load data

In [2]:
q = 'select * from bidders'
bidders = redshift_query_read(q, schema='production')

In [3]:
bidders.tail()

,ticketholder,form,product,amount,date_created
1405977,3343027,947282,84934,28750.0,2021-09-13 16:35:30
1405978,3549653,947169,84792,11700.0,2021-09-16 09:36:57
1405979,0,976397,139806,250.0,2021-09-18 09:32:28
1405980,0,976668,143045,195.0,2021-09-20 20:29:38
1405981,4101519,947282,84934,28925.0,2021-09-23 13:54:57


In [42]:
bidders['date_created'].max()

Timestamp('2021-09-23 20:16:33')

In [19]:
q = '''select 
            * 
        from transactions 
        where 
            status='A' and
            form in (select distinct form from bidders)'''
trans = redshift_query_read(q, schema='production')

In [20]:
trans[['form', 'status', 'amount', 'events_tickets']].tail()

,form,status,amount,events_tickets
66866,954277,A,0.0,1
66867,954277,A,0.0,1
66868,954277,A,50.0,1
66869,954277,A,50.0,1
66870,954277,A,100.0,0


In [43]:
trans['date'].max()

Timestamp('2022-06-19 00:00:00')

# analysis

## all

In [21]:
# get bidders counts
bidder_counts = bidders.groupby('form')['ticketholder'].nunique().reset_index()
bidder_counts.columns = ['form', 'bidders']
bidder_counts.head(2)

,form,bidders
0,947169,9508
1,947176,1


In [22]:
ticketholders = trans.groupby('form')['events_tickets'].sum().reset_index()
ticketholders.columns = ['form', 'ticketholders']
ticketholders.head(2)

,form,ticketholders
0,947201,21
1,947230,2


In [23]:
df = bidder_counts.merge(ticketholders, on='form')
df['bidders/ticketholders'] = df['bidders'] / df['ticketholders']

In [24]:
print("{:,} forms".format(len(df)))
print("{:.2f} mean bidders/ticketholders".format(df[df['ticketholders']>0]['bidders/ticketholders'].mean()))
print("{:.2f} median bidders/ticketholders".format(df[df['ticketholders']>0]['bidders/ticketholders'].median()))

549 forms
1.05 mean bidders/ticketholders
0.01 median bidders/ticketholders


## 2021+

In [27]:
# get bidders counts
bidder_counts = bidders[bidders['date_created']>='2021-01-01'].groupby('form')['ticketholder'].nunique().reset_index()
bidder_counts.columns = ['form', 'bidders']

In [31]:
ticketholders = trans[trans['date']>='2021-01-01'].groupby('form')['events_tickets'].sum().reset_index()
ticketholders.columns = ['form', 'ticketholders']

In [32]:
df = bidder_counts.merge(ticketholders, on='form')
df['bidders/ticketholders'] = df['bidders'] / df['ticketholders']

In [33]:
print("{:,} forms".format(len(df)))
print("{:.2f} mean bidders/ticketholders".format(df[df['ticketholders']>0]['bidders/ticketholders'].mean()))
print("{:.2f} median bidders/ticketholders".format(df[df['ticketholders']>0]['bidders/ticketholders'].median()))

280 forms
0.04 mean bidders/ticketholders
0.01 median bidders/ticketholders


# 2020+

In [38]:
# get bidders counts
bidder_counts = bidders[bidders['date_created']>='2020-01-01'].groupby('form')['ticketholder'].nunique().reset_index()
bidder_counts.columns = ['form', 'bidders']

In [39]:
ticketholders = trans[trans['date']>='2020-01-01'].groupby('form')['events_tickets'].sum().reset_index()
ticketholders.columns = ['form', 'ticketholders']

In [40]:
df = bidder_counts.merge(ticketholders, on='form')
df['bidders/ticketholders'] = df['bidders'] / df['ticketholders']

In [41]:
print("{:,} forms".format(len(df)))
print("{:.2f} mean bidders/ticketholders".format(df[df['ticketholders']>0]['bidders/ticketholders'].mean()))
print("{:.2f} median bidders/ticketholders".format(df[df['ticketholders']>0]['bidders/ticketholders'].median()))

529 forms
0.42 mean bidders/ticketholders
0.01 median bidders/ticketholders
